<a href="https://colab.research.google.com/github/MaheepChaudhary/GANs_mnist/blob/main/GANs_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
#importing the necessary packages
import tensorflow as tf
import tensorflow.keras as tk
from tensorflow.keras.datasets import mnist 
import imageio
import numpy as np
from sklearn.utils import shuffle

In [39]:
#preprocessing of dataset(normalization and shuffling of images)
dataset = mnist.load_data(path="mnist.npz")
x_train,y_train,x_test,y_test = dataset[0][0],dataset[0][1],dataset[1][0],dataset[1][1]
x_train = x_train/255.0;x_test = x_test/255.0
x_train,y_train = shuffle(x_train,y_train)
x_test,y_test = shuffle(x_test,y_test)

In [ ]:
#generator model
def generator()

In [ ]:
#discriminator model


In [ ]:
#loss


In [ ]:
#optimizers


In [ ]:
#generative_loss


In [ ]:
#discriminator_loss


In [ ]:
#optimizer


In [ ]:
#training_Step


In [ ]:
#training_function


In [17]:
#plotting and saving image function


In [ ]:
#training occurs
